# Step through process of data generation, salt creation, hash generation and search method for string matching in a stepped down environment.

## Protected environment
The protected environment contains data that cannot be distributed into less well controlled information or physical spaces.  Activities involving source data all occur within an environment that has the required controls for the confidentiality needs of the data, and also the management and maintenance of the salt.

In [1]:
import random, string, hashlib, json

salt_version = 0
salt_dict = []
coarse_names = {}
fine_names = {}

### Create a data set
As an example create a dataset that you wish to be able to validate against where some of the data is concrete (such as name) and some of the data has increasing fidelity such as date of birth or addresses.  Create a dataset that consists of a name (concrete) and an address (variable) based on three fields, city, street and number.  

The dataset can be any source format but needs to be reliablly extracted with strict formatting rules between search input and record storage (case, handling spaces and hyphenating, language specific characters, unknown values, date ranges etc).

Data that is not protected can also be stored unhashed to provide an indication of how you managed to match.  In this example it is the origin of the name and a reference for the entry.  An entry reference could be used to compare or extract data from other sources.

In [2]:
person_data = {'100001': ['mouse', 'mickey', 'orlando', 'main street', '30', 'DISNEY'],
        '100002': ['Duck', 'Donald', 'ducksville', '13th street', '1313', 'DISNEY'],
        '100003': ['Duck', 'Don', 'ducksville', '13th street', '1313', 'DISNEY'], # Entry represents a name alternative
        '100004': ['Kent', 'Clark', 'new york', 'unknown', 'unknown', 'MARVEL'],
        '100005': ['Kent', 'Clark', 'new york', 'clinton street', '344', 'MARVEL'], # This is a coarse clash but not fine clash
        '100006': ['Kent', 'Clark', 'metropolis', 'unknown', 'unknown', 'MARVEL'], # Entry represents alternative city
        '100007': ['Kent', 'Clark', 'new york', 'clinton street', '344', 'WARNER BROS.']} # Multiple production entries on a single person entry 

print ('{person_data_size} person records loaded.'.format(person_data_size=len(person_data)))

7 person records loaded.


### Salt Generation
Generate a salt through a sufficiently random routine and store it alongside an identifier, (date, version or other factor).  For simplicity this system only stores a single 16 character salt and attaches to a version as a key:value pair.  The salt has to be managed within the protected environment but will have to be used in stepped down environments in order to create the search query.

In [3]:
salt = ''.join(random.SystemRandom().choice(string.ascii_uppercase + string.digits + string.ascii_lowercase) for _ in range(16))
salt_version += 1
salt_dict.append({salt_version: salt})

print('Created salt {salt_string} as salt version {salt_version}.'.format(salt_string=salt, 
                                                                           salt_version=salt_version))

Created salt maDHSiTuiVd4U8ah as salt version 1.


### Generate single record data entry.
Define a schema for joining data and then store as a new data set.  In this example each row level record is constructed as two different sets to allow course and fine grained matching;
- Fine grained consists as name and full address 
- Course grained consists of name with just city

Each record is then combined with the versioned salt to prevent some common attack methods, such as dictionary attacks and rainbow tables.  The salt ultimately prevents the record from being tested with name and location data in the event it is acquired by someone wishing to validate where all the superheros live.

In [4]:
coarse_names = {}
fine_names = {}
salt_used = salt
salt_used_version = salt_version

for entry_id, name_record in person_data.items():
    # Create a string of the data to be matched against
    fine_record = '{surname}:{firstname}:{city}:{street}:{house_number}:{salt}'.format(surname=name_record[0].lower(),
                                                                              firstname=name_record[1].lower(),
                                                                              city=name_record[2],
                                                                              street=name_record[3],
                                                                              house_number=name_record[4],
                                                                              salt=salt)
    
    coarse_record = '{surname}:{firstname}:{city}:{salt}'.format(surname=name_record[0].lower(),
                                                                              firstname=name_record[1].lower(),
                                                                              city=name_record[2],
                                                                              salt=salt)
    
    # Create the hashed versions of the search strings
    fine_record_hash = hashlib.sha3_512(bytes(fine_record, 'utf-8')).hexdigest()
    coarse_record_hash = hashlib.sha3_512(bytes(coarse_record, 'utf-8')).hexdigest()
    
    # Add to a dictionary of hashes related to entry_id and the entry_type
    try:
        fine_names[fine_record_hash].append((entry_id, name_record[5]))
    except KeyError:
        fine_names[fine_record_hash] = [(entry_id, name_record[5])]
    
    try:
        coarse_names[coarse_record_hash].append((entry_id, name_record[5]))
    except KeyError:
        coarse_names[coarse_record_hash] = [(entry_id, name_record[5])]
    
print('Created fine grained ({fine_len} records) and \
coarse grained ({coarse_len} records) data \
sets using salt version {salt_used}'.format(fine_len=len(fine_names), coarse_len=len(coarse_names), salt_used=salt_used_version))

Created fine grained (6 records) and coarse grained (5 records) data sets using salt version 1


### Hashed data sets and versioned salt
Two data sets are created for distribution and represent a coarse and a fine grained exact string match capability, **without containing the original data**.  The use of the salt prevents reconstruction of the original entries by trying name and address data, something that could be done if the format of the original string construction is known and the data set is obtained.

In [5]:
# Hashing Information
print('Hashing Information:\nVersion: {salt_version} \nSalt: {salt}\n'.format(salt_version=salt_used_version, salt=salt_used))

# Original Data Set
print('Original Data Set:\n{original_data}\n'.format(original_data=json.dumps(person_data)))

# Coarse Data Set
print('Coarse Grained Data Set: \n{coarse_json}\n'.format(coarse_json=json.dumps(coarse_names)))

# Fine Grained Data Set
print('Fine Grained Data Set: \n{fine_json}'.format(fine_json=json.dumps(fine_names)))

Hashing Information:
Version: 1 
Salt: maDHSiTuiVd4U8ah

Original Data Set:
{"100001": ["mouse", "mickey", "orlando", "main street", "30", "DISNEY"], "100002": ["Duck", "Donald", "ducksville", "13th street", "1313", "DISNEY"], "100003": ["Duck", "Don", "ducksville", "13th street", "1313", "DISNEY"], "100004": ["Kent", "Clark", "new york", "unknown", "unknown", "MARVEL"], "100005": ["Kent", "Clark", "new york", "clinton street", "344", "MARVEL"], "100006": ["Kent", "Clark", "metropolis", "unknown", "unknown", "MARVEL"], "100007": ["Kent", "Clark", "new york", "clinton street", "344", "WARNER BROS."]}

Coarse Grained Data Set: 
{"51901981075923a58f77b1d6fe364d9c04fc04699cfb60d8d6e65a059f5aa3e14bb0ced2735b38a0111d625cdefa515f966ee5bd273f36fcc18fe9fca92a1caa": [["100001", "DISNEY"]], "1e296f559ad90597d7e6866552eb930f4bd0f76ac8bb1fbc1bfbeb1b0cb3d31690851b36cdbe88ba266ea6764a565eb9ad202e7c65fb7b90dd78b4ec6d1712b1": [["100002", "DISNEY"]], "c681e0d5e1aaa39be4206755daa469f62417b619a80702326c09

## Stepped Down Environment

### Salt Management
The data entering a stepped down environment consists only of hashed data.  In order to be able to construct a searchable string you will also require the salt, however, the salt needs to be protected to prevent brute force data washing.  Suggestions in a live deployment to protect the salt;
- Centralise the storage of the salt as it is extracted out from the protected environment and only distribute when you need to use it.
- Implement a symetric key encryption of the salt, enabling distribution while protecting through encryption and then break glass the symentric key.
The salt should not exist on disk within the search client, once it has been entered as part of enabling search then it should **only** be stored in memory.

### Data Distribution
The hashed data needs to be distributed to the search end point, while this could involve payload encryption in flight it may not be necessary if sufficient controls exist within the transport layer.  The data may benefit from being encrypted within the application to prevent data loss through compromise of the operating system but depends on existing controls.

In [9]:
salt_in = input('Salt to be used (default = V{last_salt_version} {last_salt}): '.format(last_salt=salt, last_salt_version=salt_version)) or salt
surname_in = input('Surname to be searched (default=Kent): ') or 'Kent'
firstname_in = input('Firstname to be searched (default=Clark): ') or 'Clark'
address_in = input('Address (house,street,city) (default=344,Clinton Street,New York): ') or '344,Clinton Street,New York'

address = address_in.split(',')
city = address[2].lower()
street = address[1].lower()
house_number = address[0]

Salt to be used (default = V1 maDHSiTuiVd4U8ah): 
Surname to be searched (default=Kent): 
Firstname to be searched (default=Clark): 
Address (house,street,city) (default=344,Clinton Street,New York): 


### Build the search string and hash
The search string has to be constructed exactly the same way as the format when it was created in the protected environment.  The salt is then added, concatenated and hashed.

In [10]:
search_fine = '{surname}:{firstname}:{city}:{street}:{house_number}:{salt}'.format(surname=surname_in.lower(),
                                                                          firstname=firstname_in.lower(),
                                                                          city=city,
                                                                          street=street,
                                                                          house_number=house_number,
                                                                          salt=salt_in)
search_coarse = '{surname}:{firstname}:{city}:{salt}'.format(surname=surname_in.lower(),
                                                           firstname=firstname_in.lower(),
                                                           city=city,
                                                           salt=salt_in)

hash_coarse = hashlib.sha3_512(bytes(search_coarse, 'utf-8')).hexdigest()
hash_fine = hashlib.sha3_512(bytes(search_fine, 'utf-8')).hexdigest()

print('Coarse Search: {coarse} \nCoarse Hash: {hash_coarse}\n \
\nFine Search: {fine} \nFine Hash: {hash_fine}\n'.format(coarse=search_coarse, 
                                                      fine=search_fine,
                                                      hash_coarse=hash_coarse,
                                                      hash_fine=hash_fine))

Coarse Search: kent:clark:new york:maDHSiTuiVd4U8ah 
Coarse Hash: 92f556172da5d3dc12a98d65e9923060e2e28f7eca57b3144703842e65efb9eaa2b2d42efd5986bfc998ed3311763a96c875859a5bc0598de6dba1850acd9b4b
 
Fine Search: kent:clark:new york:clinton street:344:maDHSiTuiVd4U8ah 
Fine Hash: 51f135bc0808b85e3ab6cc93439d00c8e0d26c0b42dc35efa034d91218e8971588789212bb68de6afaa3616ca16a890e0a3b0ed288bf372c3b11406c0c77de35



### Search Results
The hash is used to search both the coarse and fine grained hash data sets.  If an identity match is found then the entry_id and other related information is returned.  
**Note:** Matches that are exact will also appear in the coarse data set.  Could add logic to remove matching entry_ids.

In [11]:
try:
    found_coarse = coarse_names[hash_coarse]
except:
    found_coarse = 'No matches found'   
try:
    found_fine = fine_names[hash_fine]
except:
    found_fine = 'No matches found'


print('Found following exact (fine grained) matches:\n{fine_results}\n'.format(fine_results=json.dumps(found_fine)))
print('Found following broad (coarse grained) matches:\n{coarse_results}\n'.format(coarse_results=json.dumps(found_coarse)))

Found following exact (fine grained) matches:
[["100005", "MARVEL"], ["100007", "WARNER BROS."]]

Found following broad (coarse grained) matches:
[["100004", "MARVEL"], ["100005", "MARVEL"], ["100007", "WARNER BROS."]]

